In [1]:
# import library
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from PIL import Image
from datetime import date


In [2]:
# Save keys
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
MODEL = "gpt-4o-mini"
openai = OpenAI()


In [3]:
import csv
from datetime import date
def log_anatomy_to_csv(anatomy, filename, folder="anatomy_posters", csv_path="anatomy_poster_log.csv"):
    today = date.today().isoformat()
    filepath = os.path.join(folder, filename)
    row = {"anatomy": anatomy, "filename": filename, "date": today, "path": filepath}
    file_exists = os.path.isfile(csv_path)
    with open(csv_path, mode='a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["anatomy", "filename", "date", "path"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)
    print(f"📝 CSV updated: {csv_path}")


In [4]:
## List of anatomy medical
medical_anatomy = [
    "Heart", "Lung", "Kidney", "Eye", "Brain", "Bladder", "Abdomen", "Uterus",
    "Skeleton", "Muscles", "Nervous system", "Venous system", "Arterial system", "Ear", "Nose",
    "Throat", "Knee", "Ankle", "Shoulder", "Vertebrae", "Elbow", "Hand", "Foot",
    "Male genitalia", "Female genitalia", "Oral cavity", "Skin", "Head"
]


In [5]:
# define artist function
def anatomy_generator(anatomy, folder="anatomy_posters", csv_path="anatomy_poster_log.csv"):
    os.makedirs(folder, exist_ok=True)
    anatomy_clean = anatomy.lower().replace(" ", "_")
    today = date.today().isoformat()
    filename = f"popart_{anatomy_clean}_{today}.png"
    filepath = os.path.join(folder, filename)
    prompt = f"Vibrant pop-art style human anatomy of {anatomy}, with anatomical landmarks, labeled parts, and bold retro font text '{anatomy}', suitable for wall art, highly detailed, colorful, 1024x1024 resolution"
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    image = Image.open(BytesIO(image_data))
    image.save(filepath)
    log_anatomy_to_csv(anatomy=anatomy, filename=filename, folder=folder, csv_path=csv_path)
    print(f"✅ Image saved: {filepath}")
    return image


In [6]:
for anatomy in medical_anatomy:
    try:
        print(f"🧠 Generating poster for: {anatomy}")
        anatomy_generator(anatomy)
    except Exception as e:
        print(f"❌ Error for {anatomy}: {e}")
